# Part 1 - Introduction to Parcel Fabric layers

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-1---Introduction-to-Parcel-Fabric-layers" data-toc-modified-id="Part-1---Introduction-to-Parcel-Fabric-layers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Part 1 - Introduction to Parcel Fabric layers</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Anatomy-of-a-Parcel-Fabric" data-toc-modified-id="Anatomy-of-a-Parcel-Fabric-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Anatomy of a Parcel Fabric</a></span></li><li><span><a href="#The-Parcel-Fabric-FeatureLayerCollection" data-toc-modified-id="The-Parcel-Fabric-FeatureLayerCollection-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>The Parcel Fabric FeatureLayerCollection</a></span></li><li><span><a href="#Parcel-Fabric-Feature-Layers" data-toc-modified-id="Parcel-Fabric-Feature-Layers-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Parcel Fabric Feature Layers</a></span></li><li><span><a href="#The-ParcelFabricManager-object" data-toc-modified-id="The-ParcelFabricManager-object-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>The <code>ParcelFabricManager</code> object</a></span><ul class="toc-item"><li><span><a href="#List-operations-(methods)-possible-on-a-ParcelFabricManager-object" data-toc-modified-id="List-operations-(methods)-possible-on-a-ParcelFabricManager-object-1.0.4.1"><span class="toc-item-num">1.0.4.1&nbsp;&nbsp;</span>List operations (methods) possible on a <code>ParcelFabricManager</code> object</a></span></li></ul></li><li><span><a href="#API-Ref-Documentation" data-toc-modified-id="API-Ref-Documentation-1.0.5"><span class="toc-item-num">1.0.5&nbsp;&nbsp;</span>API Ref Documentation</a></span></li></ul></li></ul></li></ul></div>

### Anatomy of a Parcel Fabric

A parcel fabric stores a dataset of connected parcels or parcel network. Parcels are composed of polygon features, line features, and point features. Parcel polygons are defined by lines that store COGO dimensions from the recorded, legal document.

A Parcel Fabric consists of:

1. A geodatabase topology
2. A polygon feature class called Records for storing legal document information about a set of parcels.
3. A point feature class representing the physical, coordinated x,y,z locations on the ground.
4. A lines feature class called Connection Lines to represent dimensions between points that are not parcel boundaries.
5. Any number of parcel types

![parcel editing image](https://pro.arcgis.com/en/pro-app/latest/help/data/parcel-editing/GUID-2239DE47-1DCF-42ED-B58D-D6C4B99DCA71-web.png)

A parcel type is composed of a polygon and line feature class and can be extended to contain attributes required by an organization.
A parcel type consists of a pair of feature classes.  
- A polygon feature class representing the area of a parcel.
- A COGO enabled line feature class representing the boundary of a parcel.

The cells below will demonstrate how to access the different parts of a parcel fabric

In [1]:
from arcgis import GIS
from arcgis.features.layer import FeatureLayerCollection

In [2]:
base_server_url = (
    "https://rextapilnx02eb.mysite.com/server/rest/services/WashingtonCounty/"
)
gis = GIS(
    "https://myenterprisesite.com/portal/",
    "my_user_name",
    "my_secure_password",
    verify_cert=False,
)

# generate the enpoint urls for feature server, version management and parcel fabric
service_endpoints = ["FeatureServer", "ParcelFabricServer", "VersionManagementServer"]
service_urls = {url: base_server_url + url for url in service_endpoints}

[v for k, v in service_urls.items()]

Setting `verify_cert` to False is a security risk, use at your own risk.


['https://rextapilnx02eb.mysite.com/server/rest/services/WashingtonCounty/FeatureServer',
 'https://rextapilnx02eb.mysite.com/server/rest/services/WashingtonCounty/ParcelFabricServer',
 'https://rextapilnx02eb.mysite.com/server/rest/services/WashingtonCounty/VersionManagementServer']

### The Parcel Fabric FeatureLayerCollection

A Parcel Fabric is published as an entire set containing the feature classes and topology described above.  The FeatureLayerCollection (FLC) provides access to these items as well as the properties and capabilities of the feature service itself.

In [3]:
parcel_fabric_flc = FeatureLayerCollection(service_urls["FeatureServer"], gis)
parcel_fabric_flc.properties.capabilities

'Query,Create,Update,Delete,Uploads,Editing,Sync,ChangeTracking'

### Parcel Fabric Feature Layers

In [4]:
[
    f"Layer ID: {layer.id} - Name: {layer.name}"
    for layer in parcel_fabric_flc.properties.layers
    if layer.type == "Feature Layer"
]

['Layer ID: 1 - Name: Records',
 'Layer ID: 3 - Name: Dirty Areas',
 'Layer ID: 4 - Name: Point Errors',
 'Layer ID: 5 - Name: Line Errors',
 'Layer ID: 6 - Name: Polygon Errors',
 'Layer ID: 7 - Name: Points',
 'Layer ID: 8 - Name: Connection Lines',
 'Layer ID: 10 - Name: AdjustmentPoints',
 'Layer ID: 11 - Name: AdjustmentLines',
 'Layer ID: 12 - Name: AdjustmentVectors',
 'Layer ID: 14 - Name: Tax_PF_Lines',
 'Layer ID: 15 - Name: Tax_PF',
 'Layer ID: 17 - Name: ConveyanceDivision_PF_Lines',
 'Layer ID: 18 - Name: ConveyanceDivision_PF',
 'Layer ID: 20 - Name: Block_PF_Lines',
 'Layer ID: 21 - Name: Block_PF',
 'Layer ID: 23 - Name: SimultaneousConveyance_PF_Lines',
 'Layer ID: 24 - Name: SimultaneousConveyance_PF',
 'Layer ID: 25 - Name: Validation Point Errors',
 'Layer ID: 26 - Name: Validation Line Errors',
 'Layer ID: 27 - Name: Validation Polygon Errors']

### The `ParcelFabricManager` object

The Parcel Fabric Manager is responsible for exposing parcel management capabilities to support a variety of workflows from different clients and systems.


In [5]:
from arcgis.features._parcel import ParcelFabricManager

# Create VMS from parcel fabric FLC
version_management_service = parcel_fabric_flc.versions

with version_management_service.get("sde.DEFAULT", "read") as version:
    parcel_fabric_manager = ParcelFabricManager(
        service_urls["ParcelFabricServer"], gis, version, parcel_fabric_flc
    )

parcel_fabric_manager.properties

{
  "name": "Parcel Fabric Server",
  "type": "Map Server Extension",
  "ParcelFabricLayers": 1
}

#### List operations (methods) possible on a `ParcelFabricManager` object

In [6]:
[n for n in dir(parcel_fabric_manager) if not n.startswith("_")]

['analyze_least_squares_adjustment',
 'apply_least_squares_adjustment',
 'assign_to_record',
 'build',
 'change_type',
 'clip',
 'copy_lines_to_parcel_type',
 'create_seeds',
 'delete',
 'divide',
 'duplicate',
 'layer',
 'merge',
 'properties',
 'reassign_features_to_record',
 'reconstruct_from_seeds',
 'set_line_label_position',
 'transfer_parcel',
 'update_history']

### API Ref Documentation

- [ArcGIS Python API - Version Manager](/python/latest/api-reference/arcgis.features.managers.html#versionmanager)
- [ArcGIS Python API - Parcel Fabric Manager](/python/latest/api-reference/arcgis.features.managers.html#parcelfabricmanager)
- [ArcGIS REST API - VersionManagementServer](/rest/services-reference/enterprise/version-management-service.htm)
- [ArcGIS REST API - ParcelFabricServer](/rest/services-reference/enterprise/overview-of-parcel-fabric-sevices.htm)
- [ArcGIS Pro - Branch Versioning Scenarios](https://pro.arcgis.com/en/pro-app/latest/help/data/geodatabases/overview/branch-version-scenarios.htm)